In [139]:
# Programming Assignment 1
# Barry Hall, An Huynh, Christian Andersen
# AIT 626 Section 003
# 01/18/2026

In [141]:
# import all the necessary packages
import re
import datetime
import random
from nltk.tokenize import word_tokenize
import json

In [143]:
# # model dictionary where keywords and answers are defined.
with open('data.json', 'r') as fp:
    dic = json.load(fp)

In [145]:
dic

{'I need (.*)': ['Why do you need %s?',
  'Would it really help you to get %s?',
  'Are you sure you need %s?'],
 "Why don'?t you ([^\\?]*)\\??": ["Do you really think I don't %s?",
  'Perhaps eventually I will %s.',
  'Do you really want me to %s?'],
 "Why can'?t I ([^\\?]*)\\??": ['Do you think you should be able to %s?',
  'If you could %s, what would you do?',
  "Does it bother you that you can't %s?"],
 'I am (.*)': ['Did you come to me because you are %s?',
  'How long have you been %s?',
  'How do you feel about being %s?'],
 "I'?m (.*)": ['How does being %s make you feel?',
  'Do you enjoy being %s?',
  "Why do you tell me you're %s?",
  "Why do you think you're %s?"],
 'Are you ([^\\?]*)\\??': ['Why does it matter whether I am %s?',
  'Are we talking about you, or me?'],
 "I don't (.*)": ["Don't you really %s?",
  "Why don't you %s?",
  'Do you want to %s?'],
 'I feel (.*)': ['Good, tell more about these feelings.',
  'Do you often feel %s?',
  'When do you usually feel %s?',


In [147]:
# dictionary of pronouns to convert
"""
THe pronouns used in the user's statement can be transformed and integrated into Eliza's
response making the response more logical. You can create a dictionary for pronoun
transformations (First-perspon pronoun to second-person pronoun).
"""
pronoun_dic = {
            "you"    : "me",
            "me"     : "you",
            "yours"    : "mine",
            "mine"     : "yours",
            "your"    : "my",
            "my"     : "your",
            "you will"    : "I will",
            "I will"     : "you will",
            "you have"    : "I have",
            "I have"     : "you have",
            "you can"    : "I can",
            "I can"     : "you can",
            "am"    : "are",
            "are"     : "am",
            "was"     : "were",
            "were"    : "was",
            "i"     : "you",
            "you"    : "i",
            "i'd"     : "you would",
            "i've"    : "you have",
            "i'll"     : "you will",
            "you've"    : "I have",
            "you'll": "I will",
}



In [149]:
def replace_prons(match):
    """ Substitute pronouns to turn 2nd pov into 1st pov and 1st pov into 2nd pov, vice versa."""
    tokenizer = word_tokenize(match)

    # call the pronoun definition and replace the element in tokenizer, join the tokens, and return to ???
    for x, token in enumerate(tokenizer):
        if token in pronoun_dic:
            tokenizer[x] = pronoun_dic[token]
    return ' '.join(tokenizer)
            

In [79]:
' '.join(["You", "were", "sad"])

'You were sad'

In [151]:
"""
Takes input from the user, usually the name.  It integrates this input during the conversation.
Based on the user's input, a dictionary of Eliza's responses can be created.   Since Eliza is a 
psychotherapist, they are ver few words that have to be given special attention.
"""
def get_input(user_input):
    """Returns Eliza's response to user input. """

    for pattern, responses in dic.items():
        if pattern == "gibberish":
            continue
            
        # checks for mathing input s the key
        ans = re.search(pattern, user_input.rstrip(",.!"), re.IGNORECASE)
        # if the key mathes the input, returns (prints) the response"
        if ans:
            # the response will be randomly returned out of all the given "conversation_dict" under  ???
            response = random.choice(responses)
            # if the question in "conversation_dict" includes "%s," the response will return with the 
            # from the input
            if "%s" in response:
                match = ans[1]
                # when and if the key word include 2nd or 1st pov ('you" or "me"), it will transform
                #pov vice versa.
                response = response % (replace_prons(match))
                return response
            else:
                return response
    ## If nothing matches, pick from the gibberish set
    return random.choice(dic["gibberish"])
                


In [153]:
"""
 The algorithm stris punctuation and tokenizes words for the inputs and answer the user's
 questions accordingly.
 These tokenized words are used to frame the next question.
 If the next input is not 'quit' the chatbot enters the while loop to continue the conversation.
"""
def handle(user_name):
    """This is the main engine of chat bots.  Verify the username entered and resend the greeting """
    print("Eliza: Welcome %s.  What do you want to say today?" % user_name)
    # standard expression for bye
    end = [r'.*quit.*', r'.*bye*', r'.*exit.*']
    user_continue = True
    while user_continue :
        # check and get input name and information
        user_input = input('%s: ' % user_name)
        print()
        # convert input into lowercase
        user_input = user_input.lower()

        # check for "end" input such as 'bye or 'exit'
        for i in end:
            user_exit = re.findall(i, user_input)
            # if there is an "end" input, exit the while loop
            if len(user_exit) != 0:
                user_continue = False
        # if there is no "end" input, continue and return an appropriate response
        if user_continue:
            response = get_input(user_input)
            print("Eliza: " + response)
 

In [155]:
def greeting ():
    """ Additional features for greeting at the correct time: morning, noon, evening. """
    current_time = datetime.datetime.now()
    # if user time zone is before 12:00pm, greeting with "Good Morning. "
    if current_time.hour < 12:
        return "Good morning."
    # if user time zone is or after 12:00pm and before 6:00m, greeting with "Good afternoon."
    elif 12 <= current_time.hour < 18:
        return "Good afternoon."
    else:
        return "good evening."
        

In [157]:
def ending ():
    """ Added the ability to end chats at the correct time: morning, noon, evening. """
    current_time = datetime.datetime.now()
    # if user time zone is before 12:00pm, greeting with "Good Morning. "
    if current_time.hour < 12:
        return "great morning."
    # if user time zone is or after 12:00pm and before 6:00m, greeting with "Good afternoon."
    elif 12 <= current_time.hour < 18:
        return "great afternoon."
    else:
        return "great evening."

In [159]:
def init_conversation():
    """Start a dialog and respon by entering an additional function greeting the username and date and time. """ 
    print("Eliza: %s I'm Eliza, a psychotherapist.  Who do I have the pleaseure of talking to today?" % greeting())
    # initialize user name input
    name_input = input('User: ')
    print()
    # tokenize user name input
    tokenizer = word_tokenize(name_input.replace('.', ''))
    # use the lat word as user name if name
    if not tokenizer:
        user_name = "Patient"
    else:
        user_name = tokenizer[-1]
    return user_name




In [161]:
def main():
    """main function to start chat conversation. """
    user_name = init_conversation()
    handle(user_name)
    # end message with user name input along with extra functionality datetime ending
    print(
        "Eliza: Bye %s. Thanks for attending this meeting."
        "Have a %s" % (
            user_name, ending()))
    

In [167]:
if __name__ == '__main__':
    main()

Eliza: Good morning. I'm Eliza, a psychotherapist.  Who do I have the pleaseure of talking to today?


User:  My name is Ozlem.



Eliza: Welcome Ozlem.  What do you want to say today?


Ozlem:  I want to rule the world.



Eliza: Why do you ask that?


Ozlem:  I don't know, I crave power.



Eliza: Do you want to know , you crave power?


Ozlem:  quit



Eliza: Bye Ozlem. Thanks for attending this meeting.Have a great morning.
